In [ ]:
pip install transformers torch torchvision torchaudio accelerate datasets sentencepiece sacrebleu


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import torch
print(torch.cuda.device_count())

1


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/seamless-m4t-v2-large"  # Use the latest version
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import torch
# Define source and target language
src_lang = "hin"  # Hindi
tgt_lang = "tam"  # Tamil

# Prepare the input with language specification
input_text = "[hin] यह एक परीक्षण वाक्य है।"  # Hindi sentence
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate translation with correct target language
with torch.no_grad():
    output_tokens = model.generate(**inputs, tgt_lang=tgt_lang)

translated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print("Translated Text:", translated_text)

Translated Text: [hin] இது ஒரு சோதனை வாக்கியம்.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm
import torch

# Language codes
src_lang = "hin"  # Hindi
tgt_lang = "tam"  # Tamil

# Read and process data
with open("Hindi_Tamil_Tourism.hi", "r", encoding="utf-8") as f:
    hindi_sentences = [s.strip() for s in f.readlines()[:5000]]  

with open("Hindi_Tamil_Tourism.ta", "r", encoding="utf-8") as f:
    tamil_sentences = [s.strip() for s in f.readlines()[:5000]]

# Batch processing parameters
batch_size = 25
num_batches = 200
total_samples = batch_size * num_batches

# Results storage
all_translated = []
all_references = []
batch_bleu_scores = []

# Process in batches with progress bar
for batch_idx in tqdm(range(num_batches), desc="Processing Batches"):
    start_idx = batch_idx * batch_size
    end_idx = start_idx + batch_size
    
    # Get current batch
    hindi_batch = hindi_sentences[start_idx:end_idx]
    tamil_batch = tamil_sentences[start_idx:end_idx]
    
    # Translate batch
    translated_batch = []
    for sentence in hindi_batch:
        input_text = f"[{src_lang}] {sentence}"
        inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
        with torch.no_grad():
            output_tokens = model.generate(**inputs, tgt_lang=tgt_lang)
        translated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
        translated_batch.append(translated_text)
    
        # Calculate batch BLEU
    references = [[ref] for ref in tamil_batch]  # Wrap each reference in list
    batch_bleu = corpus_bleu(references, translated_batch)  # Direct float value
    batch_bleu_scores.append(batch_bleu)
    
    # Store results
    all_translated.extend(translated_batch)
    all_references.extend(tamil_batch)

    # Optional: Print batch progress
    tqdm.write(f"Batch {batch_idx+1}/{num_batches} - BLEU: {batch_bleu*100:.2f}")

# Calculate overall BLEU
overall_references = [[ref] for ref in all_references]
overall_bleu = corpus_bleu(overall_references, all_translated)

# Print results
print("\nBatch-wise BLEU Scores:")
for i, score in enumerate(batch_bleu_scores):
    print(f"Batch {i+1}: {score:.4f}")

print(f"\nOverall BLEU Score for {total_samples} samples: {overall_bleu:.4f}")

Processing Batches:   0%|▏                                    | 1/200 [00:28<1:34:59, 28.64s/it]

Batch 1/200 - BLEU: 48.97


Processing Batches:   1%|▎                                    | 2/200 [00:53<1:27:18, 26.46s/it]

Batch 2/200 - BLEU: 45.56


Processing Batches:   2%|▌                                    | 3/200 [01:15<1:20:16, 24.45s/it]

...

Batch 198/200 - BLEU: 46.79


Processing Batches: 100%|██████████████████████████████████▊| 199/200 [1:17:57<00:25, 25.01s/it]

Batch 199/200 - BLEU: 56.99


Processing Batches: 100%|███████████████████████████████████| 200/200 [1:18:19<00:00, 23.50s/it]


Batch 200/200 - BLEU: 51.63

Batch-wise BLEU Scores:
Batch 1: 0.4897
Batch 2: 0.4556
Batch 3: 0.4646
Batch 4: 0.4205
Batch 5: 0.5008
Batch 6: 0.4329
Batch 7: 0.2524
Batch 8: 0.2901
Batch 9: 0.5197
Batch 10: 0.4802
Batch 11: 0.4604
Batch 12: 0.4516
Batch 13: 0.4525
Batch 14: 0.4657
Batch 15: 0.4926
Batch 16: 0.4703
Batch 17: 0.4227
Batch 18: 0.4934
Batch 19: 0.4670
Batch 20: 0.5081
Batch 21: 0.3873
Batch 22: 0.4278
Batch 23: 0.4099
Batch 24: 0.2334
Batch 25: 0.2334
Batch 26: 0.2346
Batch 27: 0.2650
Batch 28: 0.2325
Batch 29: 0.2530
Batch 30: 0.1920
Batch 31: 0.2435
Batch 32: 0.2614
Batch 33: 0.2141
Batch 34: 0.2802
Batch 35: 0.2472
Batch 36: 0.2396
Batch 37: 0.2293
Batch 38: 0.2530
Batch 39: 0.2600
Batch 40: 0.2154
Batch 41: 0.2701
Batch 42: 0.2806
Batch 43: 0.2542
Batch 44: 0.2229
Batch 45: 0.1937
Batch 46: 0.2433
Batch 47: 0.2402
Batch 48: 0.1958
Batch 49: 0.2223
Batch 50: 0.2118
Batch 51: 0.1999
Batch 52: 0.2137
Batch 53: 0.2431
Batch 54: 0.2499
Batch 55: 0.2308
Batch 56: 0.2403
Batc